This notebook is a combination of all the initial OLS models ran for the state of IL. There are three seperate regression models. 

The first accounts for the distance from the population centroid of each census tract within the state of IL to the nearest optometrist facility in comparison to the socioeconomic variables (percent white population, median household income, and percent with a bachelors degree or higher). 

The second model accounts for the distance from the population centroid of each census tract within the state of IL to the nearest ophthamologist facility in comparison to the socioeconomic variables (percent white population, median household income, and percent with a bachelors degree or higher). 

The third model accounts for the distance from the population centroid of each census tract within the state of IL to the nearest 'other' vision facility in comparison to the socioeconomic variables (percent white population, median household income, and percent with a bachelors degree or higher). 

All of these models ended of violating the assumptions of OLS regression modeling and resulted in us evenutally adjusting our dependent variable. These results don't accurately represent all the conclusions of our research

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
#import geoplot as gplt
import contextily
from shapely.geometry import Point
from shapely import wkt
pd.set_option('display.max_columns', None)
import pyarrow as pyarrow
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
#import datasets
vision = pd.read_parquet("../Vision Work/vision_providers_minimal.parquet")
#county = gpd.read_file("/srv/data/my_shared_data_folder/pb-data/county.gpkg")
#zip_to_county = gpd.read_file("../data/ZIP_COUNTY_032023 (1).csv")
ILcentroids = pd.read_csv("../Geospatial/ILCensusCentroids.csv")
# link to ill censustracts centroids csv: https://www2.census.gov/geo/docs/reference/cenpop2020/tract/CenPop2020_Mean_TR17.txt

In [2]:
#subsetting vision provider data based on classification (ie optometrist/ophthamologist) and entity type
vision2 = vision[vision["Entity Type Code"] == 1]
vision2['Optometry'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('152') for code in x.split('|')))
vision2['Ophthalmology'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('207') for code in x.split('|')))
vision2['Others'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('156') for code in x.split('|')))

# creating geodataframes and subsetting data for just illinois
ILcentroids = gpd.GeoDataFrame(
   ILcentroids, geometry=gpd.points_from_xy(ILcentroids.LONGITUDE, ILcentroids.LATITUDE),
    crs="EPSG:4326")

vision3 = gpd.GeoDataFrame(
   vision2, geometry=gpd.points_from_xy(vision2.Longitude, vision2.Latitude),
    crs="EPSG:4326")
vision3 = vision3[vision3["Provider Business Mailing Address State Name"] == 'IL']

#changing crs to projected crs
ILcentroidsp = ILcentroids.to_crs("ESRI:102008")
vision3p = vision3.to_crs("ESRI:102008")

/tmp/ipykernel_2637624/4025650584.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vision2['Optometry'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('152') for code in x.split('|')))
/tmp/ipykernel_2637624/4025650584.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vision2['Ophthalmology'] = vision2['Taxonomy'].apply(lambda x: any(code.startswith('207') for code in x.split('|')))
/tmp/ipykernel_2637624/4025650584.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [3]:
# code demonstrating how to merge dataframes based on nearest
ILmerged = ILcentroidsp.sjoin_nearest(vision3p, how='inner', lsuffix='left', rsuffix='right', distance_col="Distances")
ILdistances = ILmerged.drop_duplicates(subset=['Distances'])
ILdistances

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances
0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510
1,17,1,201,2067,39.953942,-91.387764,POINT (370846.943 3590.241),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1522.315718
2,17,1,202,2870,39.941336,-91.385826,POINT (371074.990 2112.646),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,756.601345
3,17,1,400,3793,39.948935,-91.406851,POINT (369342.060 2925.819),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1240.863085
4,17,1,500,1719,39.944326,-91.398339,POINT (370052.365 2416.000),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,368.957210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3248,17,201,4002,6613,42.456448,-89.055590,POINT (536459.032 309725.343),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None,None,None,OD,None,NaN,None,None,None,None,None,None,NaN,4775 E ROCKTON RD UNIT A,None,ROSCOE,IL,61073,US,M,None,None,None,None,NaN,07/19/2021,61073,4775 E ROCKTON RD UNIT A ROSCOE IL 61073,42.455967,-89.024281,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,2421.493091
3249,17,201,4003,2102,42.488844,-89.061292,POINT (535739.372 313511.231),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None,None,None,OD,None,NaN,None,None,None,None,None,None,NaN,4775 E ROCKTON RD UNIT A,None,ROSCOE,IL,61073,US,M,None,None,None,None,NaN,07/19

In [4]:
# merging dataframes based on nearest Optometrist
vision3p_OPT = vision3p[vision3p["Optometry"]==True]
ILnearest_OPT = ILcentroidsp.sjoin_nearest(vision3p_OPT, how='inner', lsuffix='left', rsuffix='right', distance_col="Distances")
ILnearest_OPT = ILnearest_OPT.drop_duplicates(subset=['Distances'])
ILnearest_OPT

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances
0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510
1,17,1,201,2067,39.953942,-91.387764,POINT (370846.943 3590.241),85110,1780633008,1,NaN,None,None,MCDOUGALL,JOHN,H,DR.,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,2445 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,2445 BROADWAY ST QUINCY IL 62301,39.935835,-91.375903,152W00000X|152W00000X|152W00000X|_|_|_|_|_|_|_...,True,False,False,2339.294025
2,17,1,202,2870,39.941336,-91.385826,POINT (371074.990 2112.646),85110,1780633008,1,NaN,None,None,MCDOUGALL,JOHN,H,DR.,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,2445 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,2445 BROADWAY ST QUINCY IL 62301,39.935835,-91.375903,152W00000X|152W00000X|152W00000X|_|_|_|_|_|_|_...,True,False,False,1028.829249
5,17,1,600,3590,39.931661,-91.383321,POINT (371331.839 982.608),85110,1780633008,1,NaN,None,None,MCDOUGALL,JOHN,H,DR.,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,2445 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,2445 BROADWAY ST QUINCY IL 62301,39.935835,-91.375903,152W00000X|152W00000X|152W00000X|_|_|_|_|_|_|_...,True,False,False,773.730725
3,17,1,400,3793,39.948935,-91.406851,POINT (369342.060 2925.819),17382,1083711642,1,NaN,None,None,LEYENDECKERS,MERVI,HANNELE,DR.,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,926 BROADWAY ST,None,QUINCY,IL,62301,US,F,None,None,None,None,NaN,None,62301,926 BROADWAY ST QUINCY IL 62301,39.934659,-91.400487,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,1760.057699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3248,17,201,4002,6613,42.456448,-89.055590,POINT (536459.032 309725.343),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None,None,None,OD,None,NaN,None,None,None,None,None,None,NaN,4775 E ROCKTON RD UNIT A,None,ROSCOE,IL,61073,US,M,None,None,None,None,NaN,07/19/2021,61073,4775 E ROCKTON RD UNIT A ROSCOE IL 61073,42.455967,-89.024281,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,2421.493091
3249,17,201,4003,2102,42.488844,-89.061292,POINT (535739.372 313511.231),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None,None,None,OD,None,NaN,None,None,None,None,None,None,NaN,4775 E ROCKTON RD UNIT A,None,ROSCOE,IL,61073,US,M,None,None,None,None,NaN,07/1

In [5]:
#Read in convert file to have common attributes by TRACTCE
convert = pd.read_csv("../IL Work/ILTractToGeoID.csv")

In [6]:
#Drop any duplicates
convert = convert.drop_duplicates()

In [7]:
#Merge convert and the distance to nearest Optometrist dataframes
mergedTract=pd.merge(ILnearest_OPT, convert, how='inner', on= "TRACTCE")

In [8]:
mergedTract

,STATEFP,COUNTYFP,TRACTCE,POPULATION_x,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances,GEOID,POPULATION_y
0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510,17001000100,4644
1,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510,17029000100,2658
2,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510,17037000100,6737
3,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510,17095000100,3011
4,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),6717,1629155379,1,NaN,None,None,SPOONER,BRIAN,S,None,None,O.D.,None,NaN,None,None,None,None,None,None,NaN,3400 BROADWAY,QUINCY MALL,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,3400 BROADWAY QUINCY MALL QUINCY IL 62301,39.936666,-91.363201,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,819.484510,17103000100,2762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6024,17,201,4002,6613,42.456448,-89.055590,POINT (536459.032 309725.343),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None,None,None,OD,None,NaN,None,None,None,None,None,None,NaN,4775 E ROCKTON RD UNIT A,None,ROSCOE,IL,61073,US,M,None,None,None,None,NaN,07/19/2021,61073,4775 E ROCKTON RD UNIT A ROSCOE IL 61073,42.455967,-89.024281,152W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,True,False,False,2421.493091,17201004002,6613
6025,17,201,4003,2102,42.488844,-89.061292,POINT (535739.372 313511.231),117529,1407386667,1,NaN,None,None,HANSEN,RYAN,None

In [9]:
#Drop duplicate rows
mergedTract = mergedTract.drop_duplicates(subset=['GEOID'])

In [10]:
#Reset the index of the mergedTract dataframe
mergedTract= mergedTract.reset_index()

In [53]:
#Read in socioeconomic information
socio= pd.read_csv("../IL Work/socio_econ_factor_with_counts_IL_tracts.csv", low_memory=False)

In [12]:
#Set geoid in socioeconomic info to string
socio['GEOID']=socio['GEOID'].astype(str)

In [13]:
#Set Geoid in mergedTract to string
mergedTract['GEOID']=mergedTract['GEOID'].astype(str)

In [14]:
#Merge the socioeconomic and merged tract dataframes
socio_econ_factors_distance=pd.merge(socio, mergedTract, how= 'left', on= "GEOID")

In [16]:
#Drop duplicate columns
OPT_socio_econ=socio_econ_factors_distance.drop(columns = ['geometry_y', 'geometry_x'])

In [55]:
#set dependent variables
y = OPT_socio_econ['Distances']

In [56]:
#Set independent variables
x = OPT_socio_econ[['WhitePerc', 'Median_household_income', 'Bach_or_higher_perc']]

In [57]:
x = sm.add_constant(x)

In [58]:
#Run optometrist model on distance vs the socioeconomic variables
OPTmodel = sm.OLS(y, x).fit()

In [59]:
#View summary of OLS model
print(OPTmodel.summary())

                            OLS Regression Results                            
Dep. Variable:              Distances   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     227.7
Date:                Wed, 26 Jul 2023   Prob (F-statistic):          3.89e-134
Time:                        18:27:53   Log-Likelihood:                -32265.
No. Observations:                3265   AIC:                         6.454e+04
Df Residuals:                    3261   BIC:                         6.456e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    2

In [22]:
#Download to a CSV file
OPT_socio_econ.to_csv('OPT_IL_Tract_socio_econ_factors_distance.csv', index = False)

In [23]:
# merging dataframes based on nearest Ophthamologist
vision3p_OPH = vision3p[vision3p["Ophthalmology"]==True]
ILnearest_OPH = ILcentroidsp.sjoin_nearest(vision3p_OPH, how='inner', lsuffix='left', rsuffix='right', distance_col="Distances")
ILnearest_OPH = ILnearest_OPH.drop_duplicates(subset=['Distances'])
ILnearest_OPH

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances
0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,3096.472307
1,17,1,201,2067,39.953942,-91.387764,POINT (370846.943 3590.241),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,2207.715749
2,17,1,202,2870,39.941336,-91.385826,POINT (371074.990 2112.646),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,1214.596005
3,17,1,400,3793,39.948935,-91.406851,POINT (369342.060 2925.819),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,1521.499523
4,17,1,500,1719,39.944326,-91.398339,POINT (370052.365 2416.000),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,865.158810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,17,201,506,4166,42.289987,-89.017016,POINT (540881.174 290326.857),82673,1821091075,1,NaN,None,None,FOWELL,SUSAN,M,DR.,None,MD,None,NaN,None,None,None,None,None,None,NaN,1235 N MULFORD RD,SUITE 100,ROCKFORD,IL,61107,US,F,None,None,None,None,NaN,None,61107,1235 N MULFORD RD SUITE 100 ROCKFORD IL 61107,42.281331,-88.998515,207W00000X|207WX0107X|_|_|_|_|_|_|_|_|_|_|_|_|_,False,True,False,1761.740016
3188,17,201,507,3996,42.272876,-89.011011,POINT (541494.392 288344.695),82673,1821091075,1,NaN,None,None,FOWELL,SUSAN,M,DR.,None,MD,None,NaN,None,None,None,None,None,None,NaN,1235 N MULFORD RD

In [24]:
#Merge OPH with convert dataset for common column
OPHMerged =pd.merge(ILnearest_OPH, convert, how='inner', on= "TRACTCE")

In [25]:
#Drop duplicate of OPH merged
OPHMerged = OPHMerged.drop_duplicates(subset=['GEOID'])

In [26]:
#Reset the index of OPH merged
OPHMerged= OPHMerged.reset_index()

In [27]:
OPHMerged

,index,STATEFP,COUNTYFP,TRACTCE,POPULATION_x,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances,GEOID,POPULATION_y
0,0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,3096.472307,17001000100,4644
1,1,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,3096.472307,17029000100,2658
2,2,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,3096.472307,17037000100,6737
3,3,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,3096.472307,17095000100,3011
4,4,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,3096.472307,17103000100,2762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3260,6024,17,201,3809,4649,42.327232,-89.012106,POINT (540936.673 294744.059),65762,1386975712,1,NaN,None,None,STEINITZ,HUGO,R,DR.,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,3865 N MULFORD RD,None,ROCKFORD,IL,61114,US,M,None,None,None,None,NaN,None,61114,3865 N MULFORD RD ROCKFORD IL 61114,42.316858,-88.998584,207W00000X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,True,False,1612.610321,17201003809,4649
3261,6025,17,201,513,4022,42.252797,-88.989931,POINT (5

In [28]:
#Set geoid column datatype as string
OPHMerged['GEOID']=OPHMerged['GEOID'].astype(str)

In [29]:
#Merge socioeconomic information with OPH distances
socio_econ_OPH=pd.merge(socio, OPHMerged, how= 'left', on= "GEOID")

In [30]:
socio_econ_OPH

,STATEFP_x,GEOID,NAME,INTPTLAT,INTPTLON,geometry_x,Total count,Optometry_count,Ophthalmology_count,Others_count,Census Tracts,TotalPop,WhitePerc,Median_household_income,Mean_household_income,Bach_or_higher_perc,No_high_school_grad_perc,Percent_insured,index,STATEFP_y,COUNTYFP,TRACTCE,POPULATION_x,LATITUDE,LONGITUDE,geometry_y,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances,POPULATION_y
0,17.0,17001000100,1.00,39.949429,-91.352474,MULTIPOLYGON (((-91.37407799119478 39.96462398...,38,30,7,1,"Census Tract 1, Adams County, Illinois",4644,91.67,56603,71837,20.7,10.0,96.4,0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,3096.472307,4644
1,17.0,17001000201,2.01,39.956116,-91.386691,MULTIPOLYGON (((-91.37733192982496 39.96510894...,38,30,7,1,"Census Tract 2.01, Adams County, Illinois",2067,89.02,44453,61734,26.9,10.3,95.7,81,17,1,201,2067,39.953942,-91.387764,POINT (370846.943 3590.241),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,2207.715749,2067
2,17.0,17001000202,2.02,39.941736,-91.385655,MULTIPOLYGON (((-91.39204996506075 39.94675899...,35,27,7,1,"Census Tract 2.02, Adams County, Illinois",2870,85.30,60690,64593,31.4,8.6,94.7,85,17,1,202,2870,39.941336,-91.385826,POINT (371074.990 2112.646),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,1214.596005,2870
3,17.0,17001000400,4.00,39.952463,-91.409623,MULTIPOLYGON (((-91.40836592631715 39.93703898...,38,30,7,1,"Census Tract 4, Adams County, Illinois",3793,78.46,42328,47794,11.2,12.6,92.7,89,17,1,400,3793,39.948935,-91.406851,POINT (369342.060 2925.819),110014,1164583084,1,NaN,None,None,HENDRICKS,SEAN,DAVID,None,None,M.D.,None,NaN,None,None,None,None,None,None,NaN,1005 BROADWAY ST,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,10/11/2020,62301,1005 BROADWAY ST QUINCY IL 62301,39.937038,-91.399544,207W00000X|207W00000X|207W00000X|207WX0107X|_|...,False,True,False,1521.499523,3793
4,17.0,17001000500,5.00,39.943582,-91.398544,MULTIPOLYGON (((-91.40262197571184 39.95047294...,35,27,7,1,"Census Tract 5, Adams County, Illinois",1719,80.4

In [31]:
#Drop extra geometry columns
socio_econ_OPH=socio_econ_OPH.drop(columns = ['geometry_y', 'geometry_x'])

In [32]:
#Set dependent variable as distance from nearest OPH office
y2 = socio_econ_OPH['Distances']

In [33]:
#Set independent variables as demographic info
x2 = socio_econ_OPH[['WhitePerc', 'Median_household_income', 'Bach_or_higher_perc']]

In [34]:
x2 = sm.add_constant(x2)

In [35]:
#Run OLS model on distance from nearest OPH office and socioecon info
OPHModel = sm.OLS(y2, x2).fit()

In [36]:
#View OPH model summary 
print(OPHModel.summary())

                            OLS Regression Results                            
Dep. Variable:              Distances   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.307
Method:                 Least Squares   F-statistic:                     484.0
Date:                Wed, 26 Jul 2023   Prob (F-statistic):          4.22e-260
Time:                        18:21:27   Log-Likelihood:                -35119.
No. Observations:                3265   AIC:                         7.025e+04
Df Residuals:                    3261   BIC:                         7.027e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    4

In [37]:
#Turn into csv file
socio_econ_OPH.to_csv('OPH_IL_Tract_socio_econ_factors_distance.csv', index = False)

In [38]:
# merging dataframes based on nearest Other
vision3p_OTH = vision3p[vision3p["Others"]==True]
ILnearest_OTH = ILcentroidsp.sjoin_nearest(vision3p_OTH, how='inner', lsuffix='left', rsuffix='right', distance_col="Distances")
ILnearest_OTH = ILnearest_OTH.drop_duplicates(subset=['Distances'])
ILnearest_OTH

,STATEFP,COUNTYFP,TRACTCE,POPULATION,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances
0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,2652.598330
1,17,1,201,2067,39.953942,-91.387764,POINT (370846.943 3590.241),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1522.315718
2,17,1,202,2870,39.941336,-91.385826,POINT (371074.990 2112.646),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,756.601345
3,17,1,400,3793,39.948935,-91.406851,POINT (369342.060 2925.819),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1240.863085
4,17,1,500,1719,39.944326,-91.398339,POINT (370052.365 2416.000),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,368.957210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,17,201,3811,1360,42.354822,-88.997264,POINT (541842.896 298080.220),16674,1528138757,1,NaN,None,None,WHALEN,CATHERINE,J,MRS.,None,OPTICIAN,None,NaN,None,None,None,None,None,None,NaN,5970 E STATE ST,None,ROCKFORD,IL,61108,US,F,None,None,None,None,NaN,None,61108,5970 E STATE ST ROCKFORD IL 61108,42.268230,-89.000715,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,10235.856769
3246,17,201,3903,6095,42.446557,-88.967223,POINT (543359.611 309062.938),16674,1528138757,1,NaN,None,None,WHALEN,CATHERINE,J,MRS.,None,OPTICIAN,None,NaN,None,None,None,None,None,None,NaN,5970 E STATE ST,None,ROCKFORD,IL,61108,US,F,None,None,None,None,NaN,None,61108,5970 E 

In [39]:
#Merge distance from other dataframe with the convert dataframe for TRACTCE common column
OTHMerged =pd.merge(ILnearest_OTH, convert, how='inner', on= "TRACTCE")

In [40]:
#Drop duplicate rows
OTHMerged = OTHMerged.drop_duplicates(subset=['GEOID'])

In [41]:
#Reset index of OTH merged
OTHMerged= OTHMerged.reset_index()

In [42]:
OTHMerged

,index,STATEFP,COUNTYFP,TRACTCE,POPULATION_x,LATITUDE,LONGITUDE,geometry,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances,GEOID,POPULATION_y
0,0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,2652.598330,17001000100,4644
1,1,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,2652.598330,17029000100,2658
2,2,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,2652.598330,17037000100,6737
3,3,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,2652.598330,17095000100,3011
4,4,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,2652.598330,17103000100,2762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3260,6024,17,201,3811,1360,42.354822,-88.997264,POINT (541842.896 298080.220),16674,1528138757,1,NaN,None,None,WHALEN,CATHERINE,J,MRS.,None,OPTICIAN,None,NaN,None,None,None,None,None,None,NaN,5970 E STATE ST,None,ROCKFORD,IL,61108,US,F,None,None,None,None,NaN,None,61108,5970 E STATE ST ROCKFORD IL 61108,42.268230,-89.000715,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,10235.856769,17201003811,1360
3261,6025,17,201,3903,6095,42.446557,-88.967223,POINT (543359.611 309062.938),16674,1528138757,1,NaN,None,None,WHAL

In [43]:
#Set GEOID column as a string
OTHMerged['GEOID']=OTHMerged['GEOID'].astype(str)

In [44]:
#Merge distance from other df and socioeconomic dataframe
socio_econ_OTH=pd.merge(socio, OTHMerged, how= 'left', on= "GEOID")

In [45]:
socio_econ_OTH

,STATEFP_x,GEOID,NAME,INTPTLAT,INTPTLON,geometry_x,Total count,Optometry_count,Ophthalmology_count,Others_count,Census Tracts,TotalPop,WhitePerc,Median_household_income,Mean_household_income,Bach_or_higher_perc,No_high_school_grad_perc,Percent_insured,index,STATEFP_y,COUNTYFP,TRACTCE,POPULATION_x,LATITUDE,LONGITUDE,geometry_y,index_right,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Certification Date,Clean Zip,Full Address,Latitude,Longitude,Taxonomy,Optometry,Ophthalmology,Others,Distances,POPULATION_y
0,17.0,17001000100,1.00,39.949429,-91.352474,MULTIPOLYGON (((-91.37407799119478 39.96462398...,38,30,7,1,"Census Tract 1, Adams County, Illinois",4644,91.67,56603,71837,20.7,10.0,96.4,0,17,1,100,4644,39.943585,-91.362273,POINT (372954.658 2470.038),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,2652.598330,4644
1,17.0,17001000201,2.01,39.956116,-91.386691,MULTIPOLYGON (((-91.37733192982496 39.96510894...,38,30,7,1,"Census Tract 2.01, Adams County, Illinois",2067,89.02,44453,61734,26.9,10.3,95.7,81,17,1,201,2067,39.953942,-91.387764,POINT (370846.943 3590.241),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1522.315718,2067
2,17.0,17001000202,2.02,39.941736,-91.385655,MULTIPOLYGON (((-91.39204996506075 39.94675899...,35,27,7,1,"Census Tract 2.02, Adams County, Illinois",2870,85.30,60690,64593,31.4,8.6,94.7,85,17,1,202,2870,39.941336,-91.385826,POINT (371074.990 2112.646),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,756.601345,2870
3,17.0,17001000400,4.00,39.952463,-91.409623,MULTIPOLYGON (((-91.40836592631715 39.93703898...,38,30,7,1,"Census Tract 4, Adams County, Illinois",3793,78.46,42328,47794,11.2,12.6,92.7,89,17,1,400,3793,39.948935,-91.406851,POINT (369342.060 2925.819),15330,1578752853,1,NaN,None,None,DOUGHERTY,CHRIS,None,MR.,None,None,None,NaN,None,None,None,None,None,None,NaN,1 STATE AND 8TH PLZ,None,QUINCY,IL,62301,US,M,None,None,None,None,NaN,None,62301,1 STATE AND 8TH PLZ QUINCY IL 62301,39.942065,-91.395169,156FX1800X|_|_|_|_|_|_|_|_|_|_|_|_|_|_,False,False,True,1240.863085,3793
4,17.0,17001000500,5.00,39.943582,-91.398544,MULTIPOLYGON (((-91.40262197571184 39.95047294...,35,27,7,1,"Census Tract 5, Adams County, Illinois",1719,80.45,39234,46834,12.2,8.8,93.0,210,17,1,500,1719,39.9443

In [46]:
#Drop duplicate geometry columns
socio_econ_OTH=socio_econ_OTH.drop(columns = ['geometry_y', 'geometry_x'])

In [47]:
#Set distance from other locations as dependent variables
y3 = socio_econ_OTH['Distances']

In [48]:
#Set socioeconomic info as dependent variables
x3 = socio_econ_OTH[['WhitePerc', 'Median_household_income', 'Bach_or_higher_perc']]

In [49]:
x3 = sm.add_constant(x3)

In [50]:
#Run OLS model on distance to other locations and demographic info
OTHModel = sm.OLS(y3, x3).fit()

In [51]:
print(OTHModel.summary())

                            OLS Regression Results                            
Dep. Variable:              Distances   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     396.6
Date:                Wed, 26 Jul 2023   Prob (F-statistic):          1.34e-219
Time:                        18:21:44   Log-Likelihood:                -38543.
No. Observations:                3265   AIC:                         7.709e+04
Df Residuals:                    3261   BIC:                         7.712e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    1

In [52]:
#Turn df into csv
socio_econ_OTH.to_csv('OTH_IL_Tract_socio_econ_factors_distance.csv', index = False)